In [ ]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 
from hnmchallenge.data_reader import DataReader
import datetime


In [ ]:
dataset = FilterdDataset()
dr = DataReader()

In [ ]:
user_data = dr.get_filtered_full_data()

In [ ]:
user_data["t_dat"].max()

In [ ]:
last_month = user_data[(user_data["t_dat"]>="2020-09-1")]

In [ ]:
last_month[DEFAULT_USER_COL].nunique()

In [ ]:
last_month

In [ ]:
sorted_data = last_month.sort_values([DEFAULT_USER_COL, "t_dat"]).reset_index(drop=True)

In [ ]:
sorted_data

In [ ]:
sorted_data[DEFAULT_USER_COL].nunique()

In [ ]:
sorted_data["last_buy"] = sorted_data.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)

In [ ]:
hold_out=sorted_data[sorted_data["t_dat"]==sorted_data["last_buy"]]

In [ ]:
hold_out

In [ ]:
hold_out[DEFAULT_USER_COL].nunique()

In [ ]:
hold_in1=sorted_data[sorted_data["t_dat"]!=sorted_data["last_buy"]]

In [ ]:
hold_in2 = user_data[(user_data["t_dat"]<"2020-09-1")]

In [ ]:
hold_in=pd.concat([hold_in1,hold_in2],axis=0)

In [ ]:
hold_in

In [ ]:
hold_in=hold_in.drop("last_buy",axis=1)

In [ ]:
hold_out=hold_out.drop("last_buy",axis=1)

In [ ]:
hold_in

In [ ]:
hold_in = hold_in.sort_values(by=[DEFAULT_USER_COL,'t_dat'],ignore_index=True)

In [ ]:
hold_in

In [ ]:
hold_out

In [ ]:
hold_in.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "last_day_holdin.feather")
hold_out.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "last_day_holdout.feather")

In [ ]:
hold_out[DEFAULT_USER_COL].nunique()

In [ ]:
hold_in[DEFAULT_USER_COL].nunique()